## ch02 
### 基础查询与排序


In [57]:
import mysql.connector as conn
import pandas as pd
db = conn.connect(
    host='localhost',
    user='root',
    password='123456789zxcvbnm',
    database = 'shop',
)

# 创建游标对象，以便执行 SQL 语句
cursor = db.cursor(buffered=True)

In [15]:
### 进行表的创建
create_table_query = """create table products
         (product_id CHAR(4) NOT NULL,
          product_name VARCHAR(100) NOT NULL,
          product_type VARCHAR(32) NOT NULL,
          sale_price INTEGER ,
          purchase_price INTEGER ,
          regist_date DATE ,
          PRIMARY KEY (product_id))"""
cursor.execute(create_table_query)
db.commit()

In [18]:
### 内容插入
insert_data_ = '''INSERT INTO products VALUES
                                            ('0001', 'T恤衫', '衣服', 1000, 500, '2009-09-20'),
                                            ('0002', '打孔器', '办公用品', 500, 320, '2009-09-11'),
                                            ('0003', '运动T恤', '衣服', 4000, 2800, NULL),
                                            ('0004', '菜刀', '厨房用具', 3000, 2800, '2009-09-20'),
                                            ('0005', '高压锅', '厨房用具', 6800, 5000, '2009-01-15'),
                                            ('0006', '叉子', '厨房用具', 500, NULL, '2009-09-20'),
                                            ('0007', '擦菜板', '厨房用具', 880, 790, '2008-04-28'),
                                            ('0008', '圆珠笔', '办公用品', 100, NULL, '2009-11-11'); '''
cursor.execute(insert_data_)
db.commit()

##### SELECT 语句基础
###### 1.从表中选择数据
```
SELECT <列名>, ……
  FROM <表名>
 WHERE <条件表达式>;
```

In [50]:
cursor.fetchall() ### 这一步有时候很重要，当读取结果出现 bug 时
select_all = "SELECT * FROM products"
cursor.execute(select_all)

# 获取查询结果
res = cursor.fetchall()

# 将结果转换
df_data = pd.DataFrame(res, columns=cursor.column_names)

In [51]:
df_data

,product_id,product_name,product_type,sale_price,purchase_price,regist_date
0,0001,T恤衫,衣服,1000,500.0,2009-09-20
1,0002,打孔器,办公用品,500,320.0,2009-09-11
2,0003,运动T恤,衣服,4000,2800.0,None
3,0004,菜刀,厨房用具,3000,2800.0,2009-09-20
4,0005,高压锅,厨房用具,6800,5000.0,2009-01-15
5,0006,叉子,厨房用具,500,NaN,2009-09-20
6,0007,擦菜板,厨房用具,880,790.0,2008-04-28
7,0008,圆珠笔,办公用品,100,NaN,2009-11-11


In [47]:

select__part = '''SELECT product_name, product_type
                  FROM products
                  WHERE product_type = '衣服';'''
cursor.execute(select__part)

res = cursor.fetchall()
print(res)
df = pd.DataFrame(res, columns=cursor.column_names)
df

[('T恤衫', '衣服'), ('运动T恤', '衣服')]


,product_name,product_type
0,T恤衫,衣服
1,运动T恤,衣服


In [48]:
select__price_ = '''SELECT product_name, product_type, sale_price
                  FROM products
                  WHERE sale_price >= 1000; '''
cursor.execute(select__price_)

res = cursor.fetchall()
print(res)
for row in res:
    print("product_name: ", row[0])
    print("produce_type: ", row[1])
    print("sale_price: ", row[2])
df = pd.DataFrame(res, columns=cursor.column_names)
df

[('T恤衫', '衣服', 1000), ('运动T恤', '衣服', 4000), ('菜刀', '厨房用具', 3000), ('高压锅', '厨房用具', 6800)]
product_name:  T恤衫
produce_type:  衣服
sale_price:  1000
product_name:  运动T恤
produce_type:  衣服
sale_price:  4000
product_name:  菜刀
produce_type:  厨房用具
sale_price:  3000
product_name:  高压锅
produce_type:  厨房用具
sale_price:  6800


,product_name,product_type,sale_price
0,T恤衫,衣服,1000
1,运动T恤,衣服,4000
2,菜刀,厨房用具,3000
3,高压锅,厨房用具,6800


In [49]:
### 通过括号可以优先处理
'''
将查询条件原封不动地写入条件表达式，有些情况下会得到错误结果
SELECT product_name, product_type, regist_date
  FROM product
 WHERE product_type = '办公用品'
   AND regist_date = '2009-09-11' # 错误表达
    OR regist_date = '2009-09-20';
'''

select_date_ = '''SELECT product_name, product_type, regist_date
                    FROM products
                    WHERE product_type = '办公用品'
                    AND (regist_date = '2009-09-11' OR regist_date = '2009-09-20'); '''
cursor.execute(select_date_)

res = cursor.fetchall()
print(res)
for row in res:
    print("product_name: ", row[0])
    print("produce_type: ", row[1])
    print("regist_date: ", row[2])
df = pd.DataFrame(res, columns=cursor.column_names)
df

[('打孔器', '办公用品', datetime.date(2009, 9, 11))]
product_name:  打孔器
produce_type:  办公用品
regist_date:  2009-09-11


,product_name,product_type,regist_date
0,打孔器,办公用品,2009-09-11


##### 对表进行聚合


In [52]:
### 计算销售单价和进货单价的合计值
group_sum = '''SELECT SUM(sale_price), SUM(purchase_price) 
               FROM products'''
cursor.execute(group_sum)

res = cursor.fetchall()
print(res)

[(Decimal('16780'), Decimal('12210'))]


In [53]:
### 使用 pandas 
df_data[['sale_price', 'purchase_price']].sum()

sale_price        16780.0
purchase_price    12210.0
dtype: float64

In [61]:
### MAX 和 MIN 也可以用于非值型数据
max_min = '''SELECT MAX(regist_date), MIN(regist_date)
             FROM products'''
cursor.execute(max_min)

res = cursor.fetchall()
print(res)

df_max_min = pd.DataFrame(res, columns=cursor.column_names)
df_max_min

[(datetime.date(2009, 11, 11), datetime.date(2008, 4, 28))]


,MAX(regist_date),MIN(regist_date)
0,2009-11-11,2008-04-28


In [62]:
### COUNT 计数
# 计算全部数据的行数，包括 NULL 所在行
count_all = '''SELECT COUNT(*)
               FROM products'''
cursor.execute(count_all)

res = cursor.fetchall()
print(res)
df_count_all =  pd.DataFrame(res, columns=cursor.column_names)
df_count_all

[(8,)]


,COUNT(*)
0,8


In [63]:
count_part = '''SELECT COUNT(purchase_price)
               FROM products'''
cursor.execute(count_part)

res = cursor.fetchall()
print(res)
df_count_all =  pd.DataFrame(res, columns=cursor.column_names)
df_count_all

[(6,)]


,COUNT(purchase_price)
0,6


##### 对表进行分组

In [65]:
### 对表进行切分
group_by_type = '''SELECT product_type, COUNT(*)
                   FROM products
                   GROUP BY product_type;'''

cursor.execute(group_by_type)

res = cursor.fetchall()
print(res)
df_group_by_type = pd.DataFrame(res, columns=cursor.column_names)
df_group_by_type

[('衣服', 2), ('办公用品', 2), ('厨房用具', 4)]


,product_type,COUNT(*)
0,衣服,2
1,办公用品,2
2,厨房用具,4


In [67]:
count_all = '''SELECT COUNT(*)
               FROM products'''
cursor.execute(count_all)

res = cursor.fetchall()
print(res)
df_count_all =  pd.DataFrame(res, columns=cursor.column_names)
df_count_all

ProgrammingError: 1140 (42000): In aggregated query without GROUP BY, expression #1 of SELECT list contains nonaggregated column 'shop.products.product_type'; this is incompatible with sql_mode=only_full_group_by